In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.preprocessing import image

import tensorflow as tf

from os.path import expanduser
home = expanduser("~")
repodir = home + '/repos/ClusNet/'
clusterList = np.load(repodir + 'data/eROSITA_no_background/clusterList.npy')
clusterDir = repodir + 'data/eROSITA_no_background/'
GLOBDIR = clusterDir + "*.fits"

import sys
from os.path import expanduser
home = expanduser("~")

sys.path.append(home+'/repos/ClusNet/code/modules/')

from ClusNet import Cluster
from ClusNet import model as m

In [5]:
clusfpaths = Cluster.get_fpaths(k='all')
split = int(len(clusfpaths)*0.80)
train_paths = clusfpaths[:split]
test_paths = clusfpaths[split:]


In [6]:
TRAIN_LENGTH = len(train_paths)
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [9]:
x = Cluster.Cluster(fpath=train_paths[0])
input_image, input_mask = load_image_train(x)

In [10]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

@tf.function
def load_image_train(x):
    input_image = tf.image.resize(x.image.reshape(384, 384, 1), (128, 128))
    input_mask = tf.image.resize(x.segmentation_mask.reshape(384, 384, 1), (128, 128))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

def load_image_test(x):
    input_image = tf.image.resize(x.image.reshape(384, 384, 1), (128, 128))
    input_mask = tf.image.resize(x.segmentation_mask.reshape(384, 384, 1), (128, 128))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [8]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)
test = dataset['test'].map(load_image_test)

NameError: name 'dataset' is not defined